In [ ]:
!git clone https://github.com/Limm-jk/yolov5.git
!pip install -U -r yolov5/requirements.txt  # install dependencies

%cd /content/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 4263, done.
remote: Total 4263 (delta 0), reused 0 (delta 0), pack-reused 4263
Receiving objects: 100% (4263/4263), 7.30 MiB | 3.60 MiB/s, done.
Resolving deltas: 100% (2893/2893), done.
Requirement already up-to-date: Cython in /usr/local/lib/python3.6/dist-packages (from -r yolov5/requirements.txt (line 4)) (0.29.21)
     |████████████████████████████████| 11.6MB 4.1MB/s 
Requirement already up-to-date: numpy>=1.18.5 in /usr/local/lib/python3.6/dist-packages (from -r yolov5/requirements.txt (line 6)) (1.19.5)
     |████████████████████████████████| 50.4MB 64kB/s 
     |████████████████████████████████| 2.2MB 37.1MB/s 
     |████████████████████████████████| 276kB 38.6MB/s 
     |████████████████████████████████| 25.9MB 125kB/s 
     |████████████████████████████████| 10.6MB 41.2MB/s 
     |████████████████████████████████| 776.8MB 25kB/s 
     |████████████████████████████████| 12.8MB 212kB/s 
     |███████████████████████████████

/content/yolov5


In [ ]:
import os
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

from pathlib import Path
from xml.dom.minidom import parse
# xml parser
from tqdm.auto import tqdm
# 오래걸리는 반복문 시각화
from shutil import copyfile

from IPython.display import Image

In [ ]:
!mkdir -p Dataset/FaceMask/Images
!mkdir -p Dataset/FaceMask/Labels

In [ ]:
!cp -rf /content/drive/MyDrive/Colab\ Notebooks/mask_dataset/merge_image/ /content/yolov5/Dataset/FaceMask/Images

In [ ]:
!mkdir -p Dataset/images Dataset/labels

### convert annotations (from COCO .xml to YOLO format .txt)

COCO는 여러 일상 데이터셋.  
이를 이용해서 많은 detection의 발전을 이루어와서 이를 이용한 annotation을 많이 사용.

하지만 YOLO가 필요한 목표행렬 Y는 text형식.  
yml도 되긴하는데, 형식의 차이가 조금씩 존재해서 convert해서 사용하도록 하겠다.  
또한, 이 text형식이 prediction의 output과도 동일함.
```
/Dataset/FaceMask/Labels/*.txt  
[class, x, y, w, h]
```

In [ ]:
FILE_ROOT = "/content/drive/MyDrive/Colab Notebooks/mask_dataset/"
IMAGE_PATH = FILE_ROOT + "images"  
ANNOTATIONS_PATH = FILE_ROOT + "annotations"

DATA_ROOT = "/content/yolov5/Dataset/"
LABELS_ROOT = DATA_ROOT + "FaceMask/Labels"
IMAGES_ROOT = DATA_ROOT + "FaceMask/Images/images/"  

DEST_IMAGES_PATH = "images"
DEST_LABELS_PATH = "labels" 

In [ ]:
classes = ['with_mask', 'without_mask', 'mask_weared_incorrect']

In [ ]:
# x1 x2 y1 y2는 사용하기 어려움.
# detection의 표준인 x y w h로 변환
def convert_data(size, box):
    x1 = int(box[0])
    y1 = int(box[1])
    x2 = int(box[2])
    y2 = int(box[3])

    dw = np.float32(1. / int(size[0]))
    dh = np.float32(1. / int(size[1]))

    w = x2 - x1
    h = y2 - y1
    x = x1 + (w / 2)
    y = y1 + (h / 2)

    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return [x, y, w, h]

def save_file(img_jpg_file_name, size, img_box):
    save_file_name = LABELS_ROOT + '/' + img_jpg_file_name + '.txt'
    # print(save_file_name)
    file_path = open(save_file_name, "a+")
    for box in img_box:

        cls_num = classes.index(box[0])

        new_box = convert_data(size, box[1:])

        file_path.write(f"{cls_num} {new_box[0]} {new_box[1]} {new_box[2]} {new_box[3]}\n")

    file_path.flush()
    file_path.close()
    
# xml파일을 분해해서 txt로 만듦
def get_xml_data(file_path, img_xml_file):
    img_path = file_path + '/' + img_xml_file + '.xml'
    # print(img_path)

    dom = parse(img_path)
    root = dom.documentElement
    img_name = root.getElementsByTagName("filename")[0].childNodes[0].data
    img_size = root.getElementsByTagName("size")[0]
    objects = root.getElementsByTagName("object")
    img_w = img_size.getElementsByTagName("width")[0].childNodes[0].data
    img_h = img_size.getElementsByTagName("height")[0].childNodes[0].data
    img_c = img_size.getElementsByTagName("depth")[0].childNodes[0].data
    # print("img_name:", img_name)
    # print("image_info:(w,h,c)", img_w, img_h, img_c)
    img_box = []
    for box in objects:
        cls_name = box.getElementsByTagName("name")[0].childNodes[0].data
        x1 = int(box.getElementsByTagName("xmin")[0].childNodes[0].data)
        y1 = int(box.getElementsByTagName("ymin")[0].childNodes[0].data)
        x2 = int(box.getElementsByTagName("xmax")[0].childNodes[0].data)
        y2 = int(box.getElementsByTagName("ymax")[0].childNodes[0].data)
        # print("box:(c,xmin,ymin,xmax,ymax)", cls_name, x1, y1, x2, y2)
        img_jpg_file_name = img_xml_file + '.jpg'
        img_box.append([cls_name, x1, y1, x2, y2])
    # print(img_box)

    save_file(img_xml_file, [img_w, img_h], img_box)

In [ ]:
files = os.listdir(ANNOTATIONS_PATH)
for file in tqdm(files):
    # print("file name: ", file)
    file_xml = file.split(".")
    get_xml_data(ANNOTATIONS_PATH, file_xml[0])

## split Images dataset

In [ ]:
from sklearn.model_selection import train_test_split
image_list = os.listdir(IMAGES_ROOT)
train_list, test_list = train_test_split(image_list, test_size=0.2, random_state=7)
val_list, test_list = train_test_split(test_list, test_size=0.5, random_state=8)

# 파일 이름을 분류함
print('total :',len(image_list))
print('train :',len(train_list))
print('val   :',len(val_list))
print('test  :',len(test_list))

total : 853
train : 682
val   : 85
test  : 86


In [ ]:
def copy_data(file_list, img_labels_root, imgs_source, type):
    # path 체크 / 없으면 생성
    root_file = Path(DATA_ROOT + DEST_IMAGES_PATH + '/' + type)
    if not root_file.exists():
        print(f"Path {root_file} is not exit")
        os.makedirs(root_file)
        print(f"Make {root_file} Directory")

    root_file = Path(DATA_ROOT + DEST_LABELS_PATH + '/' + type)
    if not root_file.exists():
        print(f"Path {root_file} is not exit")
        os.makedirs(root_file)
        print(f"Make {root_file} Directory")


    # 분류된 파일 이름을 기준으로 소스폴더에서 파일을 타겟폴더로 복사함
    for file in file_list:
        img_name = file.replace('.png', '')
        img_src_file = imgs_source + '/' + img_name + '.png'
        label_src_file = img_labels_root + '/' + img_name + '.txt'

        # Copy image
        DICT_DIR = DATA_ROOT + DEST_IMAGES_PATH + '/' + type
        img_dict_file = DICT_DIR + '/' + img_name + '.png'

        copyfile(img_src_file, img_dict_file)

        # Copy label
        DICT_DIR = DATA_ROOT + DEST_LABELS_PATH + '/' + type
        img_dict_file = DICT_DIR + '/' + img_name + '.txt'
        copyfile(label_src_file, img_dict_file)

In [ ]:
copy_data(train_list, LABELS_ROOT, IMAGES_ROOT, "train")
copy_data(val_list,   LABELS_ROOT, IMAGES_ROOT, "val")
copy_data(test_list,  LABELS_ROOT, IMAGES_ROOT, "test")

Path /content/yolov5/Dataset/images/train is not exit
Make /content/yolov5/Dataset/images/train Directory
Path /content/yolov5/Dataset/labels/train is not exit
Make /content/yolov5/Dataset/labels/train Directory
Path /content/yolov5/Dataset/images/val is not exit
Make /content/yolov5/Dataset/images/val Directory
Path /content/yolov5/Dataset/labels/val is not exit
Make /content/yolov5/Dataset/labels/val Directory
Path /content/yolov5/Dataset/images/test is not exit
Make /content/yolov5/Dataset/images/test Directory
Path /content/yolov5/Dataset/labels/test is not exit
Make /content/yolov5/Dataset/labels/test Directory


## Create YML file
학습을 위해서 학습에 대한 정보를 제공하는 yml파일을 만들어야 한다.


1.   train : train path
2.   val : validation path
3.   nc : number of class
4.   names : 클래스 이름 배열



In [ ]:
!echo "train: Dataset/images/train\n" > data/facemask.yaml
!echo "val:   Dataset/images/val\n" >> data/facemask.yaml
!echo "nc : 3\n" >> data/facemask.yaml
!echo "names: ['With_Mask', 'Without_Mask', 'Incorrect_Mask']\n" >> data/facemask.yaml

!cat data/facemask.yaml

train: Dataset/images/train\n
val:   Dataset/images/val\n
nc : 3\n
names: ['With_Mask', 'Without_Mask', 'Incorrect_Mask']\n


In [ ]:
%cd ../
!ls

## Train
1. --data : dataset yml path
2. --cfg : yolo v5 architecture path
3. --weights : pre-trained model path

**추가사항**  
pre-trained하고 architecture뒤에 붙은 s는 뭔가요?!  
s / m / l / x 로 나누어짐.  
순서대로 무겁고 성능이 좋다.

In [ ]:
!python train.py --img 320 --batch 16 --epochs 300 --data data/facemask.yaml --cfg models/yolov5s.yaml --weights yolov5s.pt

In [ ]:
!cp yolov5/runs/train/exp/weights/last.pt .
# 최근 작업한 것을 last로 저장.
# 이중 성능이 좋았던 모델을 best로 저장

## Test
1. --conf : threshold임
2. --weights : model path
3. --save-txt : txt로 저장하겠음

In [ ]:
%cd yolov5

In [ ]:
!python3 detect.py --source Dataset/images/test --img-size 320 --conf 0.4 --weights weights/best.pt 